# Práctica 1: Algoritmo RC4
*CC. Dra. Rocío Aldeco-Pérez.*

RC4 es un cifrado de flujo famoso por ser muy simple y rápido a nivel de software. Se han encontrado varias vulnerabilidades en RC4, lo que lo hace vulnerable especialmente cuando no se descarta el comienzo del flujo de clave de salida, o cuando se utilizan claves no aleatorias o relacionadas. Por esta razón, en 2015 IETF publicó el RFC 7465 para prohibir el uso de RC4 en TLS (un protocolo muy importante utilizado por los navegadores de Internet para asegurar las comunicaciones).

RC4 genera un flujo de bits pseudoaleatorios (llamado flujo de clave). Como con cualquier cifrado de flujo, estos pueden ser utilizados para el cifrado combinándolo con el texto plano utilizando la operación XOR a nivel de bits. El descifrado se realiza de la misma manera (ya que XOR con los datos dados es una involución). Para generar el flujo de clave, el cifrado utiliza un estado interno secreto que consta de dos partes:

1. Una permutación de los 256 posibles bytes (denominada "S" a continuación).
2. Dos punteros de índice de 8 bits (denominados "i" y "j").

La permutación se inicializa con una clave de longitud variable, típicamente entre 40 y 256 bits, utilizando el algoritmo de programación de clave (KSA). Una vez que esto se ha completado, se genera el flujo de bits utilizando el algoritmo de generación pseudoaleatorio (PRGA). Luego, este flujo de bits se combina mediante XOR con el texto plano dado.

In [2]:
# Key Scheduling Algorithm en Python.

def KSA(key):
    """A partir de una clave dada genera una subclave de 256 bytes."""
    length = len(key)
    s = list(range(256))
    j = 0
    for i in range(0, 256):
        j = (j + s[i] + key[i % length]) % 256
        s[i], s[j] = s[j], s[i]
    return s

# Algoritmo de generación Pseudo-aleatoria

El byte de salida se selecciona buscando los valores de S(i) y S(j), sumándolos módulo 256 y luego utilizando la suma como índice en S; S(S(i) + S(j)) se utiliza como un byte del flujo de clave, K.

Durante tantas iteraciones como se necesiten, PRGA modifica el estado y produce un byte del flujo de clave. En cada iteración, la PRGA incrementa i, busca el elemento i-ésimo de S, S[i], y le suma j, intercambia los valores de S[i] y S[j], y luego utiliza la suma S[i] + S[j] (módulo 256) como índice para obtener un tercer elemento de S (el valor del flujo de clave K), que se XORéa con el siguiente byte del mensaje para producir el siguiente byte de cifrado o texto plano. Cada elemento de S se intercambia con otro elemento al menos una vez cada 256 iteraciones.

 ``` 
i := 0
j := 0
while GeneratingOutput:
    i := (i + 1) mod 256
    j := (j + S[i]) mod 256
    swap values of S[i] and S[j]
    K := S[(S[i] + S[j]) mod 256]
    output K
endwhile
 ``` 

 Ahora veamos su implementación en Python: